# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import udf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, sum,  asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import when, col
from pyspark.sql.window import Window
from pyspark.sql import Row
import datetime

import pyspark
import pandas as pd
import numpy as np



In [2]:
# create a Spark session
spark = SparkSession\
.builder\
.appName('Sparkify')\
.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/18 21:00:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalAccessError: class org.apache.spark.storage.StorageUtils$ (in unnamed module @0x6ced7109) cannot access class sun.nio.ch.DirectBuffer (in module java.base) because module java.base does not export sun.nio.ch to unnamed module @0x6ced7109
	at org.apache.spark.storage.StorageUtils$.<init>(StorageUtils.scala:213)
	at org.apache.spark.storage.StorageUtils$.<clinit>(StorageUtils.scala)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.<init>(BlockManagerMasterEndpoint.scala:110)
	at org.apache.spark.SparkEnv$.$anonfun$create$9(SparkEnv.scala:348)
	at org.apache.spark.SparkEnv$.registerOrLookupEndpoint$1(SparkEnv.scala:287)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:336)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:191)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:277)
	at org.apache.spark.SparkContext.<init>(SparkContext.s

In [ ]:
spark.sparkContext.getConf().getAll()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [ ]:
data = spark.read.json("/Users/Moaze002/Documents/mini_sparkify_event_data.json")

In [ ]:
data.head()

In [ ]:
data.printSchema()

In [ ]:
data.describe()


In [ ]:
data.count()

In [ ]:
data.select("userId").dropDuplicates().sort("userId").show(10)

In [ ]:
data.select("sessionId").dropDuplicates().sort("sessionId").show(10)

In [ ]:
data = data.dropna(how = 'any', subset = ["userId"])

In [ ]:
number_of_rows = data.count()

In [ ]:
data = data.filter(data["userId"] != "")

In [ ]:
# loop over each column and count duplicates
for column in data.columns:
    duplicates = data.groupBy(column).agg(count("*").alias("count")).filter("count > 1")
    duplicates.show()


In [ ]:
data_no_duplicated = data.select("*").dropDuplicates()


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
# flag the records where Cancellation Confirmation page is reached - 1 if it is and 0 if not
churn_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

#creating churn column
data_churn = data_no_duplicated.withColumn("Churn", churn_event("page"))


In [ ]:
data_pd = data_churn.toPandas()

In [ ]:
data_pd['Churn'].value_counts().to_frame().style.background_gradient()


In [ ]:
data_pd.head()

In [ ]:
print(f"Percentage of records that is omitted becuase of empty id is: {(number_of_rows - data_pd.shape[0])/data_pd.shape[0] }")

In [ ]:
data_pd[data_pd['Churn']==1].userId.count()

# Number of churn customers 😞

In [ ]:
data_pd.shape

In [ ]:
data_pd.isnull().sum()

In [ ]:
data_pd.dropna(inplace=True)

In [ ]:
data_pd.shape

In [ ]:
# create a density plot
plt.hist(data=data_pd, x='length', bins=50, density=True, alpha=0.7
        )

# add a vertical line at the median length
median_length = data_pd['length'].median()
plt.axvline(median_length, color='red', linestyle='--')

# set the x and y labels
plt.xlabel('Length')
plt.ylabel('Density')

# add a title
plt.title('Distribution of sequence lengths')

# show the plot
plt.show()

In [ ]:
df_col = data_churn.select(['userId', 'Churn', 'page']).dropDuplicates().groupBy(['Churn', 'page']).count()
df_col.show(40)

In [ ]:
# define the window specification
window = Window.partitionBy("userId") \
               .orderBy(desc("ts")) \
               .rangeBetween(Window.unboundedPreceding, 0)

# add a new column to the dataframe, called "Churn"
# this column contains the cumulative sum of the "Churn" column for each user, sorted in reverse time order
data_churn = data_churn.withColumn("Churn", sum("Churn").over(window))

In [ ]:
# get all columns except for userId and churn
columns = ['gender', 'level','page']


# create subplots
fig, axs = plt.subplots(ncols=len(columns), figsize=(18, 6))

# iterate over columns and create a bar plot for each one
for i, col in enumerate(columns):
    # group by column and churn and count
    df_col = data_churn.select(['userId', 'Churn', col]).dropDuplicates().groupBy(['Churn', col]).count()
    # collect data as a list of rows
    rows = df_col.collect()
    # convert rows to a list of dictionaries
    data = [{col: r[col], 'count': r['count'], 'Churn': r['Churn']} for r in rows]
    # set order for plotting
    data = sorted(data, key=lambda x: x['count'], reverse=True)
    # plot bar chart
    ax = sns.barplot(x=col, y='count', hue='Churn', data=pd.DataFrame(data), ax=axs[i],
                    palette=['green', 'red'])
    # set title
    ax.set_title(f'Number of Users That Churned by {col}')
    # set legend
    ax.legend(title='Churn', loc='best', labels=['Not Churned', 'Churned'])
    # set xlabel rotation
    ax.tick_params(axis='x', rotation=90)

# set overall title
fig.suptitle('Churn by User Attributes')
# adjust spacing between subplots
fig.tight_layout()


In [ ]:
group_by_page = data_churn.select(['userId', 'Churn', 'page'])\
.groupBy('page','Churn')\
.count()\
.toPandas() 

In [ ]:
# calculate counts of customers who churned and those who stayed
churn_count = group_by_page.loc[group_by_page['Churn'] == 1, 'count'].sum()
stay_count = group_by_page.loc[group_by_page['Churn'] == 0, 'count'].sum()

# calculate the rate of pages visited by those who churned vs. those who stayed
group_by_page['rate'] = group_by_page['count'] / np.where(
    group_by_page['Churn'] == 0, stay_count, churn_count)
group_by_page


In [ ]:
# plot the pages by churn
plt.figure(figsize=[10,8])
ax = sns.scatterplot(data=group_by_page, x='rate', y='page', hue='Churn', s=200)

plt.title('Rate of Pages Navigated to by Users that Churned vs. Users that Stayed')
plt.xlabel('Rate of pages visited')
plt.ylabel('Page')
plt.show()


In [ ]:
# define a UDF to extract the hour from the timestamp
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)

# create a new column for the hour
df = data_churn.withColumn("hour", get_hour(data_churn.ts))

# filter the data for users who did not churn and navigated to the NextSong page
songs_in_hour_stay = df.filter((df.page == "NextSong") )

# group by hour and count the number of NextSong pages
songs_in_hour_stay = songs_in_hour_stay.groupBy(["hour", "Churn"]).count()

# sort the data by hour as a float
songs_in_hour_stay = songs_in_hour_stay.orderBy(col("hour").cast("float"))

# convert the data to a pandas dataframe
songs_in_hour_stay_pd = songs_in_hour_stay.toPandas()

# convert the hour column to numeric data type
songs_in_hour_stay_pd["hour"] = pd.to_numeric(songs_in_hour_stay_pd["hour"])


In [ ]:
songs_in_hour_stay_pd

In [ ]:
#plot the distribution
plt.figure(figsize=[10, 8])
sns.lineplot(data=songs_in_hour_stay_pd, x='hour', y='count', hue = "Churn")
plt.xlim(-1, 24)
plt.xlabel("Hour")
plt.ylabel("Songs played")
plt.title("Number of Songs Played per Hour ( Churn & Non-Churn Users)");


This is somewhat what we expect!

In [ ]:

df_opsys = data_churn.select("userId", "userAgent", "Churn").dropDuplicates(['userId']).toPandas()

# List of browsers to extract
browser_list = ["Chrome", "Firefox", "Safari", "Trident"]

# Extract the browser from the user agent string and create a new column
df_opsys['browser'] = df_opsys.userAgent.str.extract('(?i)({0})'.format('|'.join(browser_list)))

# Group the data by OS and browser and get the count
grouped_data = df_opsys.groupby(['Churn', 'browser']).size().unstack()

# Plot a bar chart for each group
grouped_data.plot(kind='bar', layout=(2,2), figsize=(10,8))

# Show the plot
plt.show()



In [ ]:
# Subtract registration from ts and create a new column
days_df = data_churn.withColumn("diif_days", (data_churn['ts']) - (data_churn['registration']))

# Convert delta_days to days and create a new column
days_df_pd = days_df.withColumn("days", days_df['diif_days']/(1000*3600*24)).toPandas()

# # Create the violin plot
plt.figure(figsize=[8,6])
sns.violinplot(data=days_df_pd, x='Churn', y='days')
plt.title('Count of Users that Churned vs. Users that Stayed by Days Since Registering')
plt.ylabel("Days Since Registered")

# Show the plot
plt.show()



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

#### Gender

In [ ]:
data_gender = data_churn\
.select(['userId', 'gender'])\
.dropDuplicates()\
.withColumn('gender', when(col('gender') == 'M', 1)\
.otherwise(0))

In [ ]:
data_gender.show(10)

In [ ]:
from pyspark.sql.functions import dense_rank
data_level = data_churn.select(['userId', 'level', 'ts']) \
.dropDuplicates() \
.sort('userId') \
.withColumn("uptodate", dense_rank() \
.over(Window.partitionBy("userId") \
.orderBy(desc("ts")))) 

In [ ]:
data_level.show(10)

In [ ]:
data_level_f = data_level \
.filter(data_level.uptodate==1) \
.drop('ts', 'uptodate') \
.withColumn('level', when(col('level') == 'paid', 1).otherwise(0))
 

In [ ]:
data_level_f.show(10)

In [ ]:
data_artist = data_churn\
.select("userId", "artist")\
.dropDuplicates()\
.groupby("userId")\
.count()\
.withColumnRenamed("count", "number_artist_listen")



In [ ]:
data_artist.show(10)

In [ ]:
data_song = data_churn\
.select(["userId", "page"]) \
.withColumn("n_song", when(col("page") == "NextSong", 1).otherwise(0)) \
.groupBy("userId") \
.agg({'n_song': 'avg'}) \
.withColumnRenamed('avg(n_song)', 'nsong')

In [ ]:
data_song.show()

In [ ]:
data_Playlist = data_churn\
.select(["userId", "page"]) \
.withColumn("p_list", when(col("page") == "Add to Playlist", 1).otherwise(0)) \
.groupBy("userId") \
.agg({'p_list': 'count'}) \
.withColumnRenamed('count(p_list)', 'p_list')

In [ ]:
data_ad = data_churn\
.select(["userId", "page"]) \
.withColumn("ad", when(col("page") == "Roll Advert", 1).otherwise(0)) \
.groupBy("userId") \
.agg({'ad': 'sum'}) \
.withColumnRenamed('sum(ad)', 'ad')

In [ ]:
data_Playlist.show(10)

In [ ]:
data_ad.show(10)

In [ ]:
data_likes  = data_churn\
.select(["userId", "page"]) \
.withColumn("n_like", when(col("page") == "Thumbs Up", 1).otherwise(0)) \
.groupBy("userId") \
.agg({'n_like': 'sum'}) \
.withColumnRenamed('sum(n_like)', 'thumbsup')

In [ ]:
data_likes.show(10)

In [ ]:
data_dlikes =  data_churn\
.select(["userId", "page"]) \
.withColumn("n_dlike", when(col("page") == "Thumbs Down", 1).otherwise(0)) \
.groupBy("userId") \
.agg({'n_dlike': 'sum'}) \
.withColumnRenamed('sum(n_dlike)', 'thumsdown')

In [ ]:
data_dlikes.show(10)

In [ ]:
data_dgrade =  data_churn\
.select(["userId", "page"]) \
.withColumn("n_dgrade", when(col("page") == "Downgrade", 1).otherwise(0)) \
.groupBy("userId") \
.agg(sum("n_dgrade").alias("dgrade"))


In [ ]:
data_dgrade.show()

In [ ]:
data_addfriend =  data_churn\
.select(["userId", "page"]) \
.withColumn("add_f", when(col("page") == "Add Friend", 1).otherwise(0)) \
.groupBy("userId") \
.agg(count("add_f").alias("add_f"))

In [ ]:
data_addfriend.show(10)

In [ ]:
data_logout =  data_churn\
.select(["userId", "page"]) \
.withColumn("logout", when(col("page") == "Logout", 1).otherwise(0)) \
.groupBy("userId") \
.agg(sum("logout").alias("logout"))

In [ ]:
data_logout.show(10)

In [ ]:
# flag the records where Cancellation Confirmation page is reached - 1 if it is and 0 if not
churn_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

#creating churn column
data_churn = data_no_duplicated.withColumn("Churn", churn_event("page"))

# sort records for a user in reverse time order so we can add up vals in churn column
W = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

# create column churn which contains sum of churn 1s over records
data_churn = data_churn.withColumn("Churn", Fsum("Churn").over(W))

In [ ]:
data_c = data_churn\
.select("userId", "Churn")\
.dropDuplicates()\
.groupby("userId", "Churn")\
.count()\
.drop('count')

In [ ]:
df_ready = data_gender\
.join(data_level_f, ["userId"])\
.join(data_artist, ["userId"])\
.join(data_Playlist, ["userId"])\
.join(data_likes, ["userId"])\
.join(data_dlikes, ["userId"])\
.join(data_dgrade, ["userId"])\
.join(data_song, ["userId"])\
.join(data_ad , ["userId"])\
.join(data_addfriend, ["userId"])\
.join(data_logout, ["userId"])\
.join(data_c, ["userId"]) \
.dropDuplicates()


In [ ]:
df_ready.show()

In [ ]:
df_ready.printSchema()


In [ ]:
float_cols = [col(c).cast('float').alias(c) for c in df_ready.columns]

df_ready = df_ready.select(float_cols)

In [ ]:
df_ready.printSchema()

In [ ]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

# Select columns to be scaled and assemble them into a single vector column
assembler = VectorAssembler(inputCols=['gender', 'level', 'number_artist_listen', 'p_list',
                                      'thumbsup', 'thumsdown', 'dgrade', 'nsong', 'ad','Churn'],
                            outputCol='features_vec')


In [ ]:
df_assembled = assembler.transform(df_ready)

# Fit StandardScaler model on assembled data
scaler = StandardScaler(inputCol='features_vec', outputCol='scaled_features')
scaler_model = scaler.fit(df_assembled)

# Apply scaler model on assembled data
df_scaled = scaler_model.transform(df_assembled)

In [ ]:
df_scaled.show(5)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
df_scaled \
    .groupby('Churn') \
    .count() \
    .toPandas() \
    .value_counts() \
    .to_frame() \
    .style \
    .background_gradient()


In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import ClassBalancer


# Convert the 'Churn' column to a numeric label column
label_indexer = StringIndexer(inputCol='Churn', outputCol='label')


# Balance the classes
balancer = ClassBalancer(targetRatio=1.0, labelCol='label', featuresCol='scaled_features')

# Train a random forest classifier
rf = RandomForestClassifier(featuresCol='balancedFeatures', labelCol='label')

# Create a pipeline
pipeline = Pipeline(stages=[label_indexer,  balancer, rf])

# Define the parameter grid for the random forest classifier
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 30]) \
    .build()

# Define the evaluator
evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')

# Define the cross validator
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3)

# Fit the pipeline to the dataset
model = cv.fit(df)

# Make predictions on a test set
predictions = model.transform(df)

# Evaluate the model
auc = evaluator.evaluate(predictions)
print('Area Under ROC: {:.2f}'.format(auc))


In [ ]:
print(pyspark.__version__)


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.